# Calculate Zonation weights

This code was used to extract subregional values to weight indicators for Zonation 4 runs in the 2023 Blueprint

Code written by Rua Mordecai

In [1]:
import os, fnmatch
import arcpy

In [2]:
# folder that contains the clipped indicators
SourceWorkspace= r"C:\zonation\bin\indicators"

In [3]:
# Total number of pixels in the subregion mask
maskPixels = 184385461

In [4]:
# Set the workspace where the indicators are
arcpy.env.workspace = SourceWorkspace

In [5]:
# make a list of indicators 
RList = arcpy.ListRasters() 

In [6]:
# print list of indicators to make sure it looks right
print(RList)

['AtlanticCoralAndHardbottomFull.tif', 'AtlanticDeepSeaCoralRichness.tif', 'AtlanticEstuarineFishHabitatMaxE.tif', 'AtlanticMarineBirds.tif', 'AtlanticMarineMammals.tif', 'AtlanticMigratoryFishHabitat.tif', 'AtlSubregion.tif', 'A_Marine_AtlanticCoralAndHardbottomFull.tif', 'A_Marine_AtlanticDeepSeaCoralRichness.tif', 'A_Marine_AtlanticEstuarineFishHabitatMaxE.tif', 'A_Marine_AtlanticMarineBirds.tif', 'A_Marine_AtlanticMarineMammals.tif', 'A_Marine_AtlanticMigratoryFishHabitat.tif', 'A_Marine_AtlSubregion.tif', 'A_Marine_CoastalShorelineCondition.tif', 'A_Marine_EastCoastalPlainOpenPineBirds.tif', 'A_Marine_EquitableAccessToPotentialParks.tif', 'A_Marine_EstuarineCoastalCondition_znd_final.tif', 'A_Marine_FireFrequency_ZND.tif', 'A_Marine_GreatPlainsPerennialGrasslands.tif', 'A_Marine_GreenwaysAndTrails.tif', 'A_Marine_GulfCoralAndHardbottom.tif', 'A_Marine_GulfDeepSeaCoralRichness.tif', 'A_Marine_GulfMarineMammals.tif', 'A_Marine_GulfMigratoryFishConnectivity2023.tif', 'A_Marine_GulfSe

## Begin calculation of weights

In [7]:
# Create a dictionary to store the results of the weight calculations
weights = {}
for r in RList:
    weights[r] = 0

In [8]:
# Loop through all indicator rasters 
for r in RList:    
    # For each element in the attribute table
    try:
        for q in arcpy.da.SearchCursor(r, ["VALUE", "COUNT"]):
            # Only include non-zero values. We ignore zeros in the weights.
            if q[0] != 0:
                # Sum up the nonzero occurrences for each indicator value
                weights[r] += q[1]
    except RuntimeError:
        pass 
   

In [9]:
# Calculate weights based on total pixels in subregion mask
for k in weights.keys():
    # Get percent non-zero
    weights[k]=weights[k]/maskPixels
    # Adjust weight based on that percent non-zero
    # Special rule for intact habitat cores
    if k[-22:] == 'IntactHabitatCores.tif':
        if weights[k] >= 0.3:
            # No weight when >= 0.3
            weights[k] = 1
        elif weights[k] < 0.1:
            # increase weights below 0.1
            weights[k] *= 3
    # Special rule for South Atlantic forest birds
    elif k[-28:] == 'SouthAtlanticForestBirds.tif':        
        if weights[k] > 0.5:
            # No weight when > 0.5
            weights[k] *= 0.1            
        elif weights[k] < 0.1:
            # increase weights below 0.1
            weights[k] *= 0.3
        else:
            weights[k] *= 0.1
    elif weights[k] > 0.5:        
        # No weight when > 0.5        
        weights[k] = 1
    elif weights[k] < 0.1:
        # increase weights below 0.1
        weights[k] *= 3
    # print weights in a nicely formatted way    
    # if indicator has no value in a subregion the weight will be zero. Skip printing those
    if weights[k] > 0:
        # Stop python from using scientific notation for small numbers        
        # Print results in zonation format while rounding to 7 decimal places to catch small values
        print(f'{round(weights[k],8):.8f}'+' 2 1 1 0.25 '+SourceWorkspace.replace('\\','/')+'/'+k)
        #print(f'{weights[k]:.7f}'str(round(result,7))+' 2 1 1 0.25 '+SourceWorkspace.replace('\\','/')+'/'+k)
    

0.35085346 2 1 1 0.25 C:/zonation/bin/indicators/AtlanticCoralAndHardbottomFull.tif
0.25353989 2 1 1 0.25 C:/zonation/bin/indicators/AtlanticDeepSeaCoralRichness.tif
0.10686183 2 1 1 0.25 C:/zonation/bin/indicators/AtlanticEstuarineFishHabitatMaxE.tif
1.00000000 2 1 1 0.25 C:/zonation/bin/indicators/AtlanticMarineBirds.tif
1.00000000 2 1 1 0.25 C:/zonation/bin/indicators/AtlanticMarineMammals.tif
0.14994449 2 1 1 0.25 C:/zonation/bin/indicators/AtlanticMigratoryFishHabitat.tif
0.44605714 2 1 1 0.25 C:/zonation/bin/indicators/AtlSubregion.tif
0.33862182 2 1 1 0.25 C:/zonation/bin/indicators/A_Marine_AtlanticCoralAndHardbottomFull.tif
0.25353989 2 1 1 0.25 C:/zonation/bin/indicators/A_Marine_AtlanticDeepSeaCoralRichness.tif
0.00170640 2 1 1 0.25 C:/zonation/bin/indicators/A_Marine_AtlanticEstuarineFishHabitatMaxE.tif
0.46258168 2 1 1 0.25 C:/zonation/bin/indicators/A_Marine_AtlanticMarineBirds.tif
0.44802176 2 1 1 0.25 C:/zonation/bin/indicators/A_Marine_AtlanticMarineMammals.tif
0.44605